In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = ChatOpenAI(
    openai_api_key=KEY,
    model_name="openai/gpt-3.5-turbo",
    temperature=0.7
)

/tmp/ipykernel_90649/1350231263.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(
